# 完成版（未完成）　できるだけ機能を関数化するパターン

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import sys
import pandas as pd

# このプログラムで取得する Amazon・ヤフオク情報を格納するためのリスト準備
data_list = []

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(3)

# Amazon商品ページを開く
# 固定入力パターン
url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771%2Cn%3A4083271&dc&fs=true&ds=v1%3AjNxS2PZfTpTxxP1weAHzLyoVu6VNah9Uop7hosBt8fU&qid=1686742341&rnid=3895771&ref=sr_nr_n_1'
# 手動入力パターン
# print('▼ 調査対象の Amazon URL を入力してください')
# url = input()
# print('')
browser.get(url)
browser.implicitly_wait(5)

# Amazon のウィンドウハンドラの保持
amazon_window = browser.current_window_handle

# 開いた Amazon ページからヤフオク商品ページを開く関数
def open_yafuoku_page():
    # Amazon商品＋ヤフオク商品群の基点要素を取得する
    amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

    # Amazon＋ヤフオク商品群の親要素内の情報を取得する
    for amazon_elem in amazon_yafuoku_item_list:

        # Amazon商品名
        amazon_name = amazon_elem.find_element(By.XPATH, './/h2//span').text
        # print('Amazon商品名：', amazon_name)

        # Amazon購入実績
        try:
            amazon_sold_num = amazon_elem.find_element(By.XPATH, './/span[contains(text(), "購入されました")]').text
        except:
            amazon_sold_num = 'Amazon購入実績情報なし'
        # finally:
            # print('Amazon購入実績：', amazon_sold_num)
    
        # Amazon価格
        try:
            amazon_price = int(amazon_elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
        except:
            amazon_price = 'Amazon価格情報なし'
        # finally:
            # print('Amazon価格：', amazon_price, '円')

        # Amazon商品URL
        amazon_url = amazon_elem.find_element(By.XPATH, './/h2/a').get_attribute('href')
        # print('Amazon商品URL：', amazon_url)

        # ヤフオク商品群の基点要素を取得する
        yafuoku_item_list = amazon_elem.find_elements(By.XPATH, './/tbody')

        # ヤフオク商品群の親要素内の情報を取得する
        for yafuoku_elem in yafuoku_item_list:

            print('Amazon商品名：', amazon_name)
            print('Amazon価格：', amazon_price)

            # ヤフオク商品名
            try:
                yafuoku_name = yafuoku_elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
                print('ヤフオク商品名：', yafuoku_name)
            except:
                print('▲ 該当ヤフオク商品なし ▲')
                # pass

            # ヤフオク現在価格
            try:
                yafuoku_price_now = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
                print('ヤフオク現在価格：', yafuoku_price_now)
            except:
                print('▲ ヤフオク現在価格なし ▲')
                # pass

            # 現在価格での利益額計算
            profit_now = amazon_price - yafuoku_price_now
            print('A-Y現在価格差(送料考慮前)：', profit_now)

            # 原罪価格での利益率計算
            profit_rate_now = profit_now / yafuoku_price_now
            print('A-Y現在利益率(送料考慮前)：', round(profit_rate_now*100, 1), '%')

            # 現在価格での利益額/率が基準値以上ならヤフオク商品ページを開いて処理を続ける
            if profit_rate_now > 0.1: # 利益率での判定

                # ヤフオク商品ページを開く
                yafuoku_elem.find_element(By.XPATH, './/a').send_keys(Keys.ENTER)

                print('---------------------------------------------------------------------------------')
                print('開いたヤフオク商品への対応を選んでください( YES / NO / END)')
                print('　YES：商品情報を取得する')
                print('　NO ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)')
                print('　END：プログラムを終了する')
                print('---------------------------------------------------------------------------------')
                
                while True:
                    ans = input()

                    if ans == 'YES':
                        # print('この商品の情報を取得します')
                        # ヤフオク商品ページの情報取得に移る ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■
                        get_yafuoku_data()
                        break
                    elif ans == 'NO':
                        print('この商品の情報取得はスキップしました')
                        # 開いたヤフオク商品ページをアクティブにしてから閉じて Amazon ページをアクティブにする
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        break
                    elif ans == 'END':
                        print('プログラムを終了します')
                        # ヤフオク商品ページと Amazon ページを閉じてプログラムを終了する
                        handle_array = browser.window_handles
                        browser.switch_to.window(handle_array[-1])
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        browser.close()
                        sys.exit()
                    else:
                        print('YES か NO か END を入力してください')
            else:
                print('◆ 利益率が基準値未満のため情報取得をスキップしました ◆')
            
            print('')
        print('')
        
# ヤフオク商品ページから各種情報を取得する関数
def get_yafuoku_data():
    # 新たに開いたヤフオク商品ページをアクティブにする
    handle_array = browser.window_handles
    browser.switch_to.window(handle_array[-1])
    yafuoku_window = browser.current_window_handle
    # print('最新のヤフオク商品ページをアクティブにしました')

    # ヤフオク商品名を取得する
    yafuoku_name = browser.find_element(By.XPATH, '//h1').text
    print('ヤフオク商品名：', yafuoku_name)

    # 出品者の評価
    # 97％以下の場合は処理をスキップする
    seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
    print('評価：', seller_rating)

    seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))
    
    if seller_rating_int < 97:
        print('◆ 評価が低いため本商品の情報収集を終了します ◆')
        browser.close()
        browser.switch_to.window(amazon_window)
        return

    # 評価数は多い方が望ましい
    seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
    print('評価数：', seller_rating_num)

    # 匿名配送情報
    # 匿名配送希望の場合は処理をスキップする
    try:
        # 匿名配送だったらその時点で情報取得をやめて画面を閉じる
        privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
        print('◆ 匿名配送のため本商品の情報収集を終了します ◆')
        browser.close()
        browser.switch_to.window(amazon_window)
        return
    except:
        privacy = '匿名配送なし'
        print('匿名配送：', privacy)

    # ヤフオク商品ページ（アクティブウィンドウ）の URL を取得する
    yafuoku_url = browser.current_url
    print('ヤフオク商品URL：', yafuoku_url)
        
    # ヤフオク現在価格を取得する
    yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '').replace(',', '')
    print('ヤフオク現在価格：', yafuoku_price_now)

    # 後の利益計算のために int 型にしておく
    if '税込' in yafuoku_price_now:
        yafuoku_price_now_int = int(yafuoku_price_now.replace('税込', '／').replace('円）', '').split('／')[1])
    else:
        yafuoku_price_now_int = int(yafuoku_price_now.split('円')[0])
        
    # 商品の状態
    state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
    print('状態：', state)
    
    # 入札数
    bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
    print('入札数：', bidnum)

    browser.close()
    browser.switch_to.window(amazon_window)

    # 送料
    # ★
    
    # 商品説明
    # ★
    
    # 直接引き取り判定
    # ★

    # 着払い判定
    # ★

    # 配送業者、営業所止め判定
    # ★

    # 訳あり品判定
    # ★

    # オークション残り日数
    # ★

    # ヤフオク送料も加味した、現在価格での利益額/利益率判定　※ Amazon 出品送料は ResaleTrap で判定
    # ★

# 取得した情報を CSV 出力用にまとめる（出品判定の証跡として残す）関数
def store_data():
    data = {
        'Amazon商品名': amazon_name,
        'Amazon価格': amazon_price,
        'Amazon商品URL': amazon_url,
        'ヤフオク商品名': yafuoku_name,
        'ヤフオク現在価格': yafuoku_price_now,
        'ヤフオク商品URL': yafuoku_url,
        'A-Y現在利益(送料込)': profit,
        '現在利益率(送料込)': profit_rate,
        '情報取得日時': datetime.datetime.now().strftime('%Y/%m/%d %H:%M'),
        }

    data_list.append(data)
    print('data_list', data_list)

# データ格納用リストに取得した情報を CSV ファイルに出力する関数
def output():
    output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
    pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')
        
def main():
    open_yafuoku_page()
    # 取得したデータをリストに格納する
    # ■ どうやる？？？
    # store_data()
    # output()

if __name__ == '__main__':
    main()

# 完成版（未完成）　モノリシック化パターン

In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import sys
import pandas as pd

# このプログラムで取得する Amazon・ヤフオク情報を格納するためのリスト準備
data_list = []

# 格納した情報の出力先ファイル名
output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(3)

# Amazon商品ページを開く
# 固定入力パターン
# url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771%2Cn%3A4083271&dc&fs=true&ds=v1%3AjNxS2PZfTpTxxP1weAHzLyoVu6VNah9Uop7hosBt8fU&qid=1686742341&rnid=3895771&ref=sr_nr_n_1'
url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051%2Cn%3A3895771&dc&fs=true&ds=v1%3A%2F4aS563Nkvk0UGsPleVbnbegcIvY1CBz%2B04FQKejLmc&qid=1686834426&ref=sr_ex_n_1'
# 手動入力パターン
# print('▼ 調査対象の Amazon URL を入力してください')
# url = input()
# print('')
browser.get(url)
browser.implicitly_wait(5)

# Amazon のウィンドウハンドラの保持
amazon_window = browser.current_window_handle

def main():
    # Amazon商品＋ヤフオク商品群の基点要素を取得する
    amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')

    # Amazon＋ヤフオク商品群の親要素内の情報を取得する
    for amazon_elem in amazon_yafuoku_item_list:
       
        # Amazon商品名
        amazon_name = amazon_elem.find_element(By.XPATH, './/h2//span').text
        print('Amazon商品名：', amazon_name)

        # Amazon購入実績
        try:
            amazon_sold_num = amazon_elem.find_element(By.XPATH, './/span[contains(text(), "購入されました")]').text
        except:
            amazon_sold_num = 'Amazon購入実績情報なし'
        # finally:
            # print('Amazon購入実績：', amazon_sold_num)
            
        # Amazon価格
        try:
            amazon_price = int(amazon_elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
        except:
            amazon_price = 'Amazon価格情報なし'
            continue
        # finally:
            # print('Amazon価格：', amazon_price)

        # Amazon商品URL
        amazon_url = amazon_elem.find_element(By.XPATH, './/h2/a').get_attribute('href')
        # print('Amazon商品URL：', amazon_url)

        # ヤフオク商品群の基点要素を取得する
        yafuoku_item_list = amazon_elem.find_elements(By.XPATH, './/tbody')

        # ヤフオク商品群の親要素内の情報を取得する
        for yafuoku_elem in yafuoku_item_list:

            # print('Amazon商品名：', amazon_name)
            print('Amazon価格：', amazon_price)
            print('Amazon商品URL：', amazon_url)

            # ヤフオク商品名
            try:
                yafuoku_name_a = yafuoku_elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
            except:
                yafuoku_name_a = ('▲ 該当情報なし ▲')
            finally:
                print('ヤフオク商品名：', yafuoku_name_a)

            # ヤフオク現在価格
            try:
                yafuoku_price_now_a = int(yafuoku_elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
            except:
                yafuoku_price_now_a = ('▲ 該当情報なし ▲')
            finally:
                print('ヤフオク現在価格：', yafuoku_price_now_a)

            # 現在価格での利益額計算
            profit_now_a = amazon_price - yafuoku_price_now_a
            print('現在価格利益(送料考慮前)：', profit_now_a)

            # 原罪価格での利益率計算
            profit_rate_now_a = profit_now_a / yafuoku_price_now_a
            print('現在価格利益率(送料考慮前)：', round(profit_rate_now_a * 100), '%')

            # 現在価格での利益額/率が基準値以上ならヤフオク商品ページを開いて処理を続ける
            # if profit_now > 100: # 利益額での判定
            if profit_rate_now_a > 0.1: # 利益率での判定

                # ヤフオク商品ページを開いてアクティブにする
                yafuoku_elem.find_element(By.XPATH, './/a').send_keys(Keys.ENTER)
                handle_array = browser.window_handles
                browser.switch_to.window(handle_array[-1])
                yafuoku_window = browser.current_window_handle
                
                print('')
                print('■ 表示中のヤフオク商品への対応を選んでください( yes / no / end)')
                print('■ 　yes：商品情報を取得する')
                print('■ 　no ：情報取得をスキップして次の商品に移る(商品違い/匿名配送/評価不足などの場合)')
                print('■ 　end：プログラムを終了する')
                
                while True:
                    ans = input()

                    if ans == 'yes':
                        # print('この商品の情報を取得します')

                        # ヤフオク商品名を取得する
                        yafuoku_name = browser.find_element(By.XPATH, '//h1').text
                        # print('ヤフオク商品名：', yafuoku_name)

                        # 出品者の評価
                        seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
                        print('評価：', seller_rating)

                        # 97％以下の場合は処理をスキップする
                        seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))

                        if seller_rating_int < 97:
                            print('◆◆◆ 評価が低いため本商品の情報収集を終了します ◆◆◆')
                            browser.close()
                            browser.switch_to.window(amazon_window)
                            break

                        # 評価数
                        seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
                        print('評価数：', seller_rating_num)

                        # 匿名配送情報
                        try:
                            # 匿名配送希望の場合は処理をスキップする
                            privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
                            print('◆◆◆ 匿名配送のため本商品の情報取得をスキップします ◆◆◆')
                            browser.close()
                            browser.switch_to.window(amazon_window)
                            break
                        except:
                            privacy = '匿名配送なし'
                            print('匿名配送：', privacy)

                        # ヤフオク商品ページ（アクティブウィンドウ）の URL を取得する
                        yafuoku_url = browser.current_url
                        # print('ヤフオク商品URL：', yafuoku_url)

                        # ヤフオク現在価格を取得する
                        yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '').replace(',', '')
                        print('ヤフオク現在価格：', yafuoku_price_now)

                        # 後の利益計算のために int 型にしておく
                        if '税込' in yafuoku_price_now:
                            yafuoku_price_now_int = int(yafuoku_price_now.replace('税込', '／').replace('円）', '').split('／')[1])
                        else:
                            yafuoku_price_now_int = int(yafuoku_price_now.split('円')[0])

                        # 商品の状態
                        state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
                        print('状態：', state)

                        # 入札数
                        bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
                        print('入札数：', bidnum)

                        # 送料
                        # 送料の「詳細」ポップアップを開く
                        browser.find_element(By.XPATH, '//a[@id="postageDetailCurrent"]').click()
                        browser.implicitly_wait(1)
                        # 配送先から「青森県」を選ぶ
                        browser.find_element(By.XPATH, '//option[contains(text(), "青森県")]').click()
                        browser.implicitly_wait(1)

                        postage_due = browser.find_element(By.XPATH, '//dt[contains(text(), "送料負担")]/following-sibling::dd').text
                        # print('送料負担：', postage_due)

                        # 配送情報を取得する
                        try:
                            shipping = browser.find_element(By.XPATH, '//dt[@class="BidModal__postageName"]').text
                            print('配送方法(詳細画面)：', shipping)

                            postage = browser.find_element(By.XPATH, '//div[@class="BidModal__postagePrice"]').text
                            print('送料(詳細画面)：', postage)
                            
                            if postage == '無料':
                                postage_int = 0
                            else:
                                postage_int = int(postage.replace('円（税込）', ''))
                            # print('送料(int)：', postage_int)
                        except:
                            pass

                        # 「✕」をクリックしてポップアップを閉じる
                        browser.find_element(By.XPATH, '//div[@class="BidModal__box BidModal__box--postage js-modal-box"]//a[contains(text(), "閉じる")]').click()

                        postage_from = browser.find_element(By.XPATH, '//p[@class="Price__postageFrom"]').text
                        print(postage_from)

                        try:
                            send_date = browser.find_element(By.XPATH, '//dt[contains(text(), "発送開始")]/following-sibling::dd').text
                        except:
                            send_date = '情報なし'
                        finally:
                            print('発送開始：', send_date)

                        try:
                            postage_area = browser.find_element(By.XPATH, '//span[@class="Price__postageValue"]').text
                        except:
                            postage_area = '情報なし'
                        finally:
                            print('送料(地域込)：', postage_area)

                        # 商品説明
                        yafuoku_info = browser.find_element(By.XPATH, '//div[@class="ProductExplanation__commentArea"]').text
                        # print('商品説明：', yafuoku_info)

                        # 直接引き取り判定
                        print('直接引取(商品名)：', re.findall('.*直接.*引.*', yafuoku_name))
                        print('直接引取(商品説明)：', re.findall('.*直接.*引.*', yafuoku_info))
                        print('手渡し(商品説明)', re.findall('.*手渡.*', yafuoku_info))

                        # 着払い判定
                        print('送料負担：', postage_due)
                        print('着払(商品名)：', re.findall('.*着払.*', yafuoku_name))
                        print('着払(商品説明)：', re.findall('.*着払.*', yafuoku_info))

                        # 配送業者、営業所止め判定
                        print('ヤマト(商品説明)：', re.findall('.*ヤマト.*', yafuoku_info))
                        print('らくらく家財便(商品説明)：', re.findall('.*らくらく.*', yafuoku_info))
                        print('佐川(商品説明)', re.findall('.*佐川.*', yafuoku_info))
                        print('日本郵便(商品説明)', re.findall('.*日本郵便.*', yafuoku_info))
                        print('日本郵便(商品説明)', re.findall('.*ゆう.*', yafuoku_info))
                        print('営業所止め(商品説明)', re.findall('.*止め.*', yafuoku_info))

                        # 訳あり品判定
                        print('訳あり(商品名)：', re.findall('.*訳.*', yafuoku_name))
                        print('訳あり(商品説明)：', re.findall('.*訳.*', yafuoku_info))

                        # オークション残り日数
                        auc_timeleft = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[1].text
                        print('残り日数(画面表示)：', auc_timeleft)

                        auc_enddatetime = browser.find_element(By.XPATH, '//th[contains(text(), "終了日時")]/following-sibling::td').text
                        print('終了予定(画面表示)：', auc_enddatetime)

                        auc_enddate = auc_enddatetime.split('（')[0]
                        auc_endtime = auc_enddatetime.split('）')[1]
                        auc_end = auc_enddate + ' ' + auc_endtime
                        auc_enddatetime = datetime.datetime.strptime(auc_end, '%Y.%m.%d %H:%M')
                        # print('終了予定(datetime型)：', auc_enddatetime)

                        dt_now = datetime.datetime.now()
                        # print('現在時刻：', dt_now.strftime('%Y-%m-%d %H:%M:%S'))

                        auc_time_left = auc_enddatetime - dt_now
                        # print('残り時間(詳細計算)：', auc_time_left)

                        # ヤフオク送料も加味した、現在価格での利益額/利益率判定　※ Amazon 出品送料は ResaleTrap で判定
                        profit_now_final = amazon_price - yafuoku_price_now_int - postage_int
                        print('現在価格利益(ヤフオク送料込 / Amazon出品手数料考慮前)：', profit_now_final, '円')

                        profit_rate_now_final = round(profit_now_final / yafuoku_price_now_int * 100)
                        print('現在価格利益率(ヤフオク送料込 / Amazon出品手数料考慮前)：', profit_rate_now_final, '%')

                        # 出品（監視対象に追加する）に値するかを判定し、合格の場合は出品する
                        print('')
                        print('■ この商品を出品候補にしますか？( yes / no )')

                        while True:
                            sell_flag = input()

                            if sell_flag == 'yes':
                                print('出品画面に進みます')
                                # 「監視対象に追加する」をクリックする。ResaleTrap のページが開かれる
                                browser.find_element(By.XPATH, '//a[contains(text(), "監視対象に追加する")]').click()

                                # ResaleTrap ページをアクティブにする
                                handle_array = browser.window_handles
                                browser.switch_to.window(handle_array[-1])

                                # 送料
                                browser.find_elements(By.XPATH, '//input')[1].send_keys(Keys.CONTROL,"a")
                                browser.find_elements(By.XPATH, '//input')[1].send_keys(postage_int)

                                print('')
                                print('■ この商品を出品しますか？')
                                print('■ 　出品する → 各項目を手動で入力し、出品が完了したら done と入力してください')
                                print('■ 　やめる　 → cancel と入力してください')
                                while True:
                                    sell_done_flag = input()

                                    if sell_done_flag == 'done':
                                        # 取得したデータをファイル出力するためにリストに追加する
                                        data = {
                                            'Amazon商品名': amazon_name,
                                            'Amazon価格[円]': amazon_price,
                                            'Amazon商品URL': amazon_url,
                                            'ヤフオク商品名': yafuoku_name,
                                            '状態': state,
                                            'ヤフオク現在価格[円]': yafuoku_price_now_int,
                                            '送料(地域込)': postage_area,
                                            '匿名配送': privacy,
                                            '評価': seller_rating,
                                            '評価数': seller_rating_num,
                                            'ヤフオク商品URL': yafuoku_url,
                                            '現在価格利益(送料込)[円]': profit_now_final,
                                            '現在価格利益率(送料込)[%]': profit_rate_now_final,
                                            '残り日数(画面表示)': auc_timeleft,
                                            '発送開始': send_date,
                                            '情報取得日時': datetime.datetime.now().strftime('%Y/%m/%d %H:%M'),
                                            }
                                        data_list.append(data)

                                        # 取得した情報を CSV ファイルに出力する。ファイル名(csv_file_name)はプログラム冒頭で定義済
                                        pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')
                                        print('取得情報をファイルに出力しました')
                                        break
                                    elif sell_done_flag == 'cancel':
                                        break
                                    else:
                                        print('done か cancel と入力してください')

                                # ResaleTrap ページを閉じる
                                browser.close()

                                # ヤフオク商品ページを閉じる
                                browser.switch_to.window(yafuoku_window)
                                browser.close()

                                # Amazon ページをアクティブにする
                                browser.switch_to.window(amazon_window)
                                break
                            elif sell_flag == 'no':
                                print('この商品の出品をスキップします')
                                # browser.switch_to.window(handle_array[-1])
                                browser.close()

                                browser.switch_to.window(amazon_window)
                                break
                            else:
                                print('yes か no を入力してください')
                        break
                    elif ans == 'no':
                        print('この商品の情報取得はスキップしました')
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        break
                    elif ans == 'end':
                        print('プログラムを終了します')
                        browser.close()
                        browser.switch_to.window(amazon_window)
                        browser.close()
                        sys.exit()
                    else:
                        print('yes か no か end を入力してください')
            else:
                print('◆◆◆ 利益率が基準値未満のため情報取得をスキップしました ◆◆◆')
            
            print('===== 次のヤフオク商品に移ります =====')
            print('')
            time.sleep(1)

        print('===== 次の Amazon 商品に移ります =====')
        print('')
        time.sleep(1)

if __name__ == '__main__':
    main()

C:\Users\amuza\AppData\Local\Temp\ipykernel_8980\766756911.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)


Amazon商品名： Takistar 低温調理器具 真空調理器 スロークッカー コンパクト IPX7防水 レシピ付 日本正規品 国内品アフターサポート プレミアム低温調理器 Sous vide ホワイト
===== 次の Amazon 商品に移ります =====

Amazon商品名： キッチン 家電 調理家電 キッチン家電 AINX スマートオートクッカー WJ0864
===== 次の Amazon 商品に移ります =====

Amazon商品名： キッチン 家電 調理家電 キッチン家電 【特典付き】アラジン グラファイトミニグリラー 721006
===== 次の Amazon 商品に移ります =====

Amazon商品名： タイガー魔法瓶(TIGER) 炊飯器 5.5合 マイコン 調理メニュー付き 炊きたて ホワイト JBH-G101W
Amazon価格： 7504
Amazon商品URL： https://www.amazon.co.jp/%E3%82%BF%E3%82%A4%E3%82%AC%E3%83%BC-%E3%83%9E%E3%82%A4%E3%82%B3%E3%83%B3%E5%BC%8F-%E7%82%8A%E3%81%8D%E3%81%9F%E3%81%A6-5-5%E5%90%88-JBH-G101W/dp/B01HC98W74/ref=sr_1_4?qid=1686853601&s=kitchen&sr=1-4
ヤフオク商品名： タイガー魔法瓶(TIGER) 炊飯器 5.5合 マイコン 調理メニュー付き 炊きたて ホワイト JBH-G101W(中古品)
ヤフオク現在価格： 13431
現在価格利益(送料考慮前)： -5927
現在価格利益率(送料考慮前)： -44 %
◆◆◆ 利益率が基準値未満のため情報取得をスキップしました ◆◆◆
===== 次のヤフオク商品に移ります =====

Amazon価格： 7504
Amazon商品URL： https://www.amazon.co.jp/%E3%82%BF%E3%82%A4%E3%82%AC%E3%83%BC-%E3%83%9E%E3%82%A4%E3%82%B3%E3%83%B3%E5%BC%8F-%E7%82%8A%E3%81%8D%E3%81%9F%E3%81%A6-

 yes


評価： 98.5%
評価数： 13,373
匿名配送： 匿名配送なし
ヤフオク現在価格： 5900円（税込6490円）
状態： やや傷や汚れあり
入札数： 0件
配送方法(詳細画面)： ゆうパック
送料(詳細画面)： 770円（税込）
発送元：滋賀県
発送開始： 情報なし
送料(地域込)： 青森県は770円（税込）（離島を除く）
直接引取(商品名)： []
直接引取(商品説明)： []
手渡し(商品説明) []
送料負担： 落札者
着払(商品名)： []
着払(商品説明)： []
ヤマト(商品説明)： []
らくらく家財便(商品説明)： []
佐川(商品説明) []
日本郵便(商品説明) []
日本郵便(商品説明) []
営業所止め(商品説明) []
訳あり(商品名)： []
訳あり(商品説明)： []
残り日数(画面表示)： 6日
終了予定(画面表示)： 2023.06.22（木）03:49
現在価格利益(ヤフオク送料込 / Amazon出品手数料考慮前)： 244 円
現在価格利益率(ヤフオク送料込 / Amazon出品手数料考慮前)： 4 %

■ この商品を出品候補にしますか？( yes / no )


 yes


...

 yes


評価： 99.5%
評価数： 2,163
匿名配送： 匿名配送なし
ヤフオク現在価格： 760円（税0円）
状態： やや傷や汚れあり
入札数： 0件
配送方法(詳細画面)： 宅急便（ヤマト運輸）
送料(詳細画面)： 送料未定
発送元：静岡県
発送開始： 支払い手続きから2～3日で発送
送料(地域込)： 落札者負担
直接引取(商品名)： []
直接引取(商品説明)： ['・直接取引に関しましては、対応致しかねます。']
手渡し(商品説明) []
送料負担： 落札者
着払(商品名)： []
着払(商品説明)： []
ヤマト(商品説明)： ['こちらの商品はヤマト運輸で発送となります【80cm】サイズとなります。', '配送会社\u3000【ヤマト運輸・佐川急便・西濃運輸】']
らくらく家財便(商品説明)： []
佐川(商品説明) ['配送会社\u3000【ヤマト運輸・佐川急便・西濃運輸】']
日本郵便(商品説明) []
日本郵便(商品説明) []
営業所止め(商品説明) []
訳あり(商品名)： []
訳あり(商品説明)： []
残り日数(画面表示)： 1日
終了予定(画面表示)： 2023.06.17（土）21:16
現在価格利益(ヤフオク送料込 / Amazon出品手数料考慮前)： 1990 円
現在価格利益率(ヤフオク送料込 / Amazon出品手数料考慮前)： 262 %

■ この商品を出品候補にしますか？( yes / no )


 yes


出品画面に進みます


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (1189, -3145)
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00308893+48451]
	(No symbol) [0x0029B8A1]
	(No symbol) [0x001A5058]
	(No symbol) [0x001D4BA4]
	(No symbol) [0x001D36E8]
	(No symbol) [0x001D1EEB]
	(No symbol) [0x001D12FE]
	(No symbol) [0x001C9AAC]
	(No symbol) [0x001EA2BC]
	(No symbol) [0x001C9586]
	(No symbol) [0x001EA614]
	(No symbol) [0x001FC482]
	(No symbol) [0x001EA0B6]
	(No symbol) [0x001C7E08]
	(No symbol) [0x001C8F2D]
	GetHandleVerifier [0x00568E3A+2540266]
	GetHandleVerifier [0x005A8959+2801161]
	GetHandleVerifier [0x005A295C+2776588]
	GetHandleVerifier [0x00392280+612144]
	(No symbol) [0x002A4F6C]
	(No symbol) [0x002A11D8]
	(No symbol) [0x002A12BB]
	(No symbol) [0x00294857]
	BaseThreadInitThunk [0x75AE00C9+25]
	RtlGetAppContainerNamedObjectPath [0x775A7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x775A7B1E+238]
	(No symbol) 

# 関数間での変数の値を受け渡すテスト

In [ ]:
def outer():
    def inner():
        a = 2
        b = a + 1
        return b

    b = inner() # bという変数を作り、戻り値を渡す。

    c = str(b)
    print(c)

outer()

In [ ]:
def open_yafuoku_page_test():
    print('run open_yafuoku_page_test()')
    amazon = 'amazon_name'
    get_yafuoku_data_test(amazon)
    
def get_yafuoku_data_test(amazon):
    print('run get_yafuoku_data_test()')
    print('name:', amazon) # OK
    yafuoku = 'yafuoku_name'
    store_data_test(amazon, yafuoku)
    
def store_data_test(amazon, yafuoku):
    print('run store_data_test()')
    print(amazon)
    print(yafuoku)

def output_test():
    print('run output_test()')
    print(amazon)
    print(yafuoku)

def main_test():
    print('run main_test()')
    open_yafuoku_page_test()
    
main_test()
# store_data_test()
output_test()